In [ ]:
# Jupter notebook magic
%matplotlib inline

# Import libraries to work with the data

import os
import sys
import importlib
from glob import glob
from datetime import datetime, timedelta
import h5py
from dask import compute
import dask
from distributed import Client
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import scipy.optimize
from PyAstronomy import pyasl
from scipy import signal
from shapely.geometry import (Polygon, Point, box)

import pyepsg

from matplotlib import pyplot as plt

import himatpy, himatpy.GRACE_MASCON.pygrace, himatpy.MAR.utils
importlib.reload(himatpy)
importlib.reload(himatpy.GRACE_MASCON.pygrace)
importlib.reload(himatpy.MAR.utils)
from himatpy.GRACE_MASCON.pygrace import __aggregate_mascon
from himatpy.GRACE_MASCON.pygrace import (extract_grace, get_mascon_gdf, trend_analysis, get_cmwe_trend_analysis, select_mascons, aggregate_mascons)
from himatpy.MAR.utils import get_xr_dataset

from himatpy.LIS import utils as LISutils
from himatpy.MAR import utils as MARutils
from himatpy.tools import animation_tools

import warnings
warnings.filterwarnings('ignore')

In [ ]:
agg_fns = sorted(glob('agg_MAR/*.nc'))
ds = xr.open_mfdataset(agg_fns,autoclose=True,concat_dim='time')

In [ ]:
masked_gdf = gpd.read_file('MAR_mascons.shp')

Grace_fn = 'Data/Grace/GSFC.glb.200301_201607_v02.4-ICE6G.h5'
# ---> use local copy
grace_file = os.path.abspath(Grace_fn)
f = extract_grace(grace_file,printGroups=False)
    
mascon = f['mascon']
soln = f['solution']
time = f['time']


In [ ]:
mascons = ds['mascons'].data
t = pd.DataFrame(data = {
    'time': f['time']["ref_days_middle"][0,:]
})
grc_lst = []
for m in mascons:
    df = pd.DataFrame(data={
        'mmwe': soln['cmwe'][m][:] * 10,
        'mascon': m
    })
    df.index = t['time'].apply(lambda x: datetime(2001,12,31) + timedelta(days=x))
    grc_lst.append(df)
    
gracedf = pd.concat(grc_lst)

In [ ]:
grct  = gracedf.groupby('time').mean()
mardf = ds.to_dataframe()
#mds   = ds.mean(dim='mascons')

In [ ]:
mdf = mardf['SMB_ice'].groupby('time').mean().cumsum().to_frame()
marwe = mardf['mmWE'].groupby('time').mean().to_frame()

In [ ]:
fig, ax = plt.subplots(1, figsize=(10, 8))

tstart = '2003-01-07'
#tstart = '2004-03-01'
pmdf = mdf.loc[tstart:'2015-12-31']
pgrc = grct.loc[tstart:'2015-12-31']
(-pmdf+pmdf.iloc[0]).plot(y='SMB_ice',ax=ax)
#xticks1 = ax.get_xticks()
#(marwe-marwe.loc[tstart]).plot(ax=ax,y='mmWE')
(pgrc-pgrc.iloc[0]).plot(ax=ax,y='mmwe',marker='x')
xticks2 = ax.get_xticks()
#plt.xticks(df.index,pmdf.index.values)
#grct.plot(ax=ax,y='mmwe',marker='x')
#(grct - grct.iloc[0].mmwe).plot(y='mmwe', linewidth = 3,ax=ax)
#(mdf - mdf.loc['2003-01-01'].mmWE).plot(ax=ax,y='mmWE',linewidth=3)
plt.ylabel('mm water', fontsize = 20)
plt.xlabel('time', fontsize = 20)
#ax.legend(['MAR_SMB','MAR_mmWE','GRACE'],fontsize = 20, loc = 1)
ax.legend(['MAR_SMB','GRACE'],fontsize = 20, loc = 1)

In [ ]:
plt.plot(aaa[:-1]-bbb)

In [ ]:
ttt = np.array([datetime(2000,1,1) + timedelta(days=x) for x in aaa]) 
